<a href="https://colab.research.google.com/github/pv21287/CE888/blob/master/Sport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the libraries needed

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import joblib

# Load the dataset

In [ ]:
df=pd.read_csv('classification.csv')

In [ ]:
df.head()

,age,interest,success
0,23.657801,18.859917,0.0
1,22.573729,17.969223,0.0
2,32.553424,29.463651,0.0
3,6.718035,25.704665,1.0
4,14.401919,16.770856,0.0


In [ ]:
df['age'].describe()

count    297.000000
mean      23.709718
std        8.864000
min        2.347560
25%       17.002148
50%       23.283644
75%       30.661149
max       45.773729
Name: age, dtype: float64

In [ ]:
df['interest'].describe()

count    297.000000
mean      46.986527
std       21.956447
min       -0.852800
25%       28.689067
50%       45.695926
75%       65.171876
max       97.175508
Name: interest, dtype: float64

In [ ]:
df['success'].value_counts()

1.0    169
0.0    128
Name: success, dtype: int64

In [ ]:
len(df)

297

In [ ]:
df.columns

Index(['age', 'interest', 'success'], dtype='object')

# Create the training and test sets

In [ ]:
X=df.drop(['success'], axis=1)
y=df['success']
X_train, X_test, y_train, y_test = train_test_split(df.drop(['success'], axis=1), df['success'], 
                                                    test_size=0.3, random_state=42, stratify=y)

# Feature Scaling

In [ ]:
from sklearn.impute import SimpleImputer


imp=SimpleImputer() 
imp_X_train = pd.DataFrame(imp.fit_transform(X_train))
imp_X_test = pd.DataFrame(imp.transform(X_test))


imp_X_train.columns = X_train.columns
imp_X_test.columns = X_test.columns

In [ ]:
X = pd.concat([imp_X_train, imp_X_test])
y = pd.concat([y_train, y_test])


In [ ]:
len(X.age)

297

# Train a Random Forest Classifier

In [ ]:
classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42, min_samples_split=10)
classifier.fit(imp_X_train, y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=10,
                       n_estimators=200, random_state=42)

In [ ]:
accuracy_score(y, classifier.predict(X))

0.9663299663299664

In [ ]:
accuracy_score(y_train, classifier.predict(imp_X_train))

0.9758454106280193

In [ ]:
accuracy_score(y_test, classifier.predict(imp_X_test))

0.9444444444444444

# Obtain the Probabilities for classes

In [ ]:
prediction_probs=classifier.predict_proba(X)

In [ ]:
len(prediction_probs)

297

In [ ]:
len(prediction_probs[0])

2

In [ ]:
df['probs_1']=[prediction_probs[i][1] for i in range(len(df.success))]

In [ ]:
df['probs_1']=[df['probs_1'][i]>=0.5 for i in range(len(df.success))]*1

In [ ]:
df.probs_1

0       True
1       True
2      False
3       True
4      False
       ...  
292     True
293     True
294     True
295     True
296    False
Name: probs_1, Length: 297, dtype: bool

In [ ]:
df.probs_1.value_counts()

True     167
False    130
Name: probs_1, dtype: int64

# Decision Tree Classifier trained on the obtained results from RF model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# y_tr=df.probs_1[y_train.index]
# y_ts=df.probs_1[y_test.index]


dt=DecisionTreeClassifier(max_depth=None, min_samples_leaf=1)

dt.fit(X, df.probs_1)


DecisionTreeClassifier()

In [ ]:
accuracy_score(y, dt.predict(X))

0.9663299663299664

In [ ]:
accuracy_score(y_train, dt.predict(imp_X_train))

0.9758454106280193

In [ ]:
accuracy_score(y_test, dt.predict(imp_X_test))

0.9444444444444444

In [ ]:
accuracy_score(df.probs_1, dt.predict(X))

1.0

In [ ]:
accuracy_score(y, dt.predict(X))

0.9663299663299664

In [ ]:
dt.predict(imp_X_test)

array([ True,  True, False, False,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True, False, False, False,  True, False, False,
        True,  True,  True, False, False, False,  True, False,  True,
        True,  True,  True, False,  True, False,  True, False, False,
       False,  True,  True, False,  True,  True,  True, False, False,
        True, False,  True, False, False,  True, False,  True,  True,
        True,  True, False, False, False,  True, False, False,  True,
        True,  True, False, False,  True, False, False, False, False,
        True, False, False,  True,  True,  True,  True,  True, False])

In [ ]:
accuracy_score(y_train, dt.predict(imp_X_train))

0.9758454106280193

# Decision Tree Classifier trained on the original data

In [ ]:
dt1=DecisionTreeClassifier(max_depth=None, min_samples_leaf=1)

dt1.fit(imp_X_train, y_train)

DecisionTreeClassifier()

In [ ]:
accuracy_score(y_train, dt1.predict(imp_X_train))

1.0

In [ ]:
accuracy_score(y_test, dt1.predict(imp_X_test))

0.8888888888888888

In [ ]:
df.head()

,age,interest,success,probs_1
0,23.657801,18.859917,0.0,True
1,22.573729,17.969223,0.0,True
2,32.553424,29.463651,0.0,False
3,6.718035,25.704665,1.0,True
4,14.401919,16.770856,0.0,False


In [ ]:
X

,age,interest
0,22.037690,70.170483
1,18.308514,54.004888
2,31.452201,33.088326
3,17.891597,87.565843
4,8.395444,15.224779
...,...,...
85,35.642075,34.763747
86,16.247653,87.729036
87,18.400920,52.789281
88,17.810775,54.469648
